In [2]:
import eumdac
import datetime
import shutil

In [3]:
consumer_key = '9dcHtMpRngCEnHBtZX4iOZL7s5Qa'
consumer_secret = 'FfJCsfRfPPOThUvOYfBEFkPzxCwa'
credentials = (consumer_key, consumer_secret)
token = eumdac.AccessToken(credentials)
print(f"This token '{token}' expires {token.expiration}")

This token 'e3b96e4a-cbca-35c4-84ce-4ff85157875c' expires 2024-06-13 16:14:24.768149


In [8]:
datastore = eumdac.DataStore(token)
for collection in datastore.collections:
    print(collection)

EO:EUM:DAT:MSG:CLM-IODC
EO:EUM:DAT:0394
EO:EUM:DAT:0412
EO:EUM:DAT:0237
EO:EUM:DAT:0241
EO:EUM:DAT:0857
EO:EUM:DAT:0862
EO:EUM:DAT:0882
EO:EUM:DAT:METOP:ASCSZF1B
EO:EUM:DAT:METOP:ASCSZR1B
EO:EUM:DAT:MSG:MSG15-RSS
EO:EUM:DAT:0855
EO:EUM:DAT:0836
EO:EUM:CM:METOP:ASCSZFR02
EO:EUM:DAT:0081
EO:EUM:DAT:0398
EO:EUM:DAT:0405
EO:EUM:DAT:0406
EO:EUM:DAT:0410
EO:EUM:DAT:0413
EO:EUM:DAT:0415
EO:EUM:DAT:0416
EO:EUM:DAT:0239
EO:EUM:DAT:0301
EO:EUM:DAT:0617
EO:EUM:DAT:0880
EO:EUM:DAT:0856
EO:EUM:DAT:METOP:ASCSZO1B
EO:EUM:DAT:METOP:GOMEL1
EO:EUM:DAT:0647
EO:EUM:DAT:0851
EO:EUM:DAT:0835
EO:EUM:DAT:0833
EO:EUM:DAT:0858
EO:EUM:DAT:0859
EO:EUM:DAT:0839
EO:EUM:DAT:0841
EO:EUM:DAT:0834
EO:EUM:DAT:0142
EO:EUM:DAT:0238
EO:EUM:DAT:0374
EO:EUM:DAT:0601
EO:EUM:DAT:0556
EO:EUM:DAT:0557
EO:EUM:DAT:0576
EO:EUM:DAT:0579
EO:EUM:DAT:0581
EO:EUM:DAT:0583
EO:EUM:DAT:0300
EO:EUM:DAT:0407
EO:EUM:DAT:0411
EO:EUM:DAT:0417
EO:EUM:DAT:0274
EO:EUM:DAT:0082
EO:EUM:DAT:0584
EO:EUM:DAT:MULT:HIRSL1
EO:EUM:DAT:METOP:GLB-SST-NC
EO:E

In [9]:
selected_collection = datastore.get_collection('EO:EUM:DAT:0410')

In [16]:
# Add vertices for polygon, wrapping back to the start point.

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

geometry = [[min_lon, min_lat],[min_lon, max_lat],[max_lon, max_lat],[max_lon, min_lat],[min_lon, min_lat]]
# Set sensing start and end time
start = datetime.datetime(2024, 6, 10)
end = datetime.datetime(2024, 6, 11)
# Retrieve datasets that match our filter
products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
    
print(f'Found Datasets: {products.total_results} datasets for the given time range')
for product in products:
    print(str(product))

for product in products:
    with product.open() as fsrc, \
        open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {product} finished.')
    print('All downloads are finished.')

Found Datasets: 10 datasets for the given time range
S3B_OL_1_ERR____20240610T132506_20240610T140932_20240611T164421_2666_094_067______MAR_O_NT_002.SEN3
S3B_OL_1_ERR____20240610T132506_20240610T140932_20240610T154354_2666_094_067______MAR_O_NR_002.SEN3
S3A_OL_1_ERR____20240610T122258_20240610T130724_20240611T150204_2666_113_209______MAR_O_NT_002.SEN3
S3A_OL_1_ERR____20240610T122258_20240610T130724_20240610T144324_2666_113_209______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240610T114407_20240610T122833_20240611T150618_2666_094_066______MAR_O_NT_002.SEN3
S3B_OL_1_ERR____20240610T114407_20240610T122833_20240610T140538_2666_094_066______MAR_O_NR_002.SEN3
S3A_OL_1_ERR____20240610T104159_20240610T112625_20240611T132759_2666_113_208______MAR_O_NT_002.SEN3
S3A_OL_1_ERR____20240610T104159_20240610T112625_20240610T130628_2666_113_208______MAR_O_NR_002.SEN3
S3B_OL_1_ERR____20240610T100308_20240610T104734_20240611T132212_2666_094_065______MAR_O_NT_002.SEN3
S3B_OL_1_ERR____20240610T100308_20240610T104734

ProtocolError: ('Connection broken: IncompleteRead(576473727 bytes read, 161279670 more expected)', IncompleteRead(576473727 bytes read, 161279670 more expected))